In [ ]:
import time
from pynq import Overlay, MMIO

# Load overlay
ol = Overlay("/home/xilinx/pynq/overlays/spi_test/design_1_wrapper.bit")

# SPI block base address and size (from your output)
base_addr = 0x41E00000
addr_range = 0x10000  # 64K range is fine

spi_mmio = MMIO(base_addr, addr_range)


import time

class Max7219:
    def __init__(self, spi_mmio):
        self.spi_mmio = spi_mmio
        self.reset()
        self.init_display()

    def reset(self):
        # Software reset
        self.spi_mmio.write(0x40, 0xA)
        # Enable SPI: Master mode, CPOL=0, CPHA=0
        self.spi_mmio.write(0x60, 0x6)
        # Select slave 0
        self.spi_mmio.write(0x70, 0x0)

    def _write(self, reg, data):
        self.spi_mmio.write(0x68, reg)
        while not (self.spi_mmio.read(0x64) & (1 << 2)):
            pass
        self.spi_mmio.write(0x68, data)
        while not (self.spi_mmio.read(0x64) & (1 << 2)):
            pass

    def init_display(self, brightness=0x08):
        """Initialize the MAX7219 display."""
        self._write(0x0F, 0x00)      # Test mode: off
        self._write(0x09, 0x00)      # Decode mode: none
        self._write(0x0B, 0x07)      # Scan limit: 8 digits
        self._write(0x0C, 0x01)      # Shutdown: normal operation
        self._write(0x0A, brightness & 0x0F)  # Intensity: 0x00 to 0x0F
        self.clear()

    def clear(self):
        """Clear all rows."""
        for row in range(1, 9):
            self._write(row, 0x00)

    def rotate_frame_90(self, frame):
        rotated = []
        for col in range(8):
            val = 0
            for row in range(8):
                bit = (frame[row] >> col) & 0x01
                val |= bit << (7 - row)
            rotated.append(val)
        return rotated

    def draw(self, frame):
        """Draw an 8-byte frame."""
        for row in range(1, 9):
            value = frame[row - 1] if row - 1 < len(frame) else 0x00
            self._write(row, value)

    def blink(self, frame1, frame2, times=5, delay=0.3):
        """Blink between two frames."""
        for _ in range(times):
            self.draw(frame1)
            time.sleep(delay)
            self.draw(frame2)
            time.sleep(delay)
        self.draw(frame1)



# Example: set up driver
led = Max7219(spi_mmio)

# Your eye frames
eye_open = [
    0b00000000,
    0b00000000,
    0b00100100,
    0b00100100,
    0b00000000,
    0b00000000,
    0b00000000,
    0b00000000,
]

eye_blink = [
    0b00000000,
    0b00000000,
    0b00000000,
    0b00100100,
    0b00000000,
    0b00000000,
    0b00000000,
    0b00000000,
]

print("Init Complete!")

In [ ]:
eye_open_rotated = led.rotate_frame_90(eye_open)
led.draw(eye_open_rotated)
eye_blink_rotated = led.rotate_frame_90(eye_blink)
led.draw(eye_open_rotated)
print("Eye pattern displayed!")

In [ ]:
# Blink!
led.blink(eye_open_rotated, eye_blink_rotated, times=5)

print("Eye blink displayed!")